# Agent Using smolagents

In the last section, we learned how we can create Agents from scratch using Python code, and we saw just how tedious that process can be. Fortunately, many Agent libraries simplify this work by handling much of the heavy lifting for you.

In this tutorial, you’ll create your very first Agent capable of performing actions such as image generation, web search, time zone checking and much more!

You will also publish your agent on a Hugging Face Space so you can share it with friends and colleagues.

Let’s get started!

## What is smolagents?

`smolagents` is a library that provides a framework for developing your agents with ease. It's designed for simplicity and abstracts away much of the complexity of building an Agent.

**In short, `smolagents` is a library that focuses on codeAgent, a kind of agent that performs “Actions” through code blocks, and then “Observes” results by executing the code.**

In [19]:
from dotenv import load_dotenv
import os
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, InferenceClientModel, load_tool, tool
import datetime
import requests
import pytz
import yaml
import gradio

load_dotenv()
token = os.getenv("HF_INF_TOKEN")

We will directly use the CodeAgent class from `smolagents`.

**The Tools**

In [20]:
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: # it's important to specify the return type
    # Keep this format for the tool description / args description but feel free to modify the tool
    """A tool that does nothing yet 
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

So far we have:
1. A non-working dummy Tool that you can modify to make something useful.
1. An actually working Tool that gets the current time somewhere in the world.

To define your tool it's important to:
1. Provide input and output types for your function
1. A well formatted docstring. `smolagents` is expecting all the arguments to have a textual description in the docstring.

## The Agent

It uses Qwen/Qwen2.5-Coder-32B-Instruct as the LLM engine. This is a very capable model that we’ll access via the serverless API.

In [22]:
final_answer = FinalAnswerTool()
model = InferenceClientModel(
    max_tokens=2096,
    temperature=0.5,
    model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    custom_role_conversions=None,
)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
# We're creating our CodeAgent
agent = CodeAgent(
    model=model,
    tools=[final_answer], # add your tools here (don't remove final_answer)
    max_steps=6,
    verbosity_level=1,
    #grammar=None, I think the example is outdated. Says "grammer is an unecpected argument". Probably can fix with the full example
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

GradioUI(agent).launch()

AssertionError: Some prompt templates are missing from your custom `prompt_templates`: {'final_answer'}

Due to issues with the dependencies present in the actual template, I'm stopping here and resuming on the template.